<a href="https://colab.research.google.com/github/rajitha-1701/Answer-Type-Prediction/blob/main/QA_Type_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df = pd.read_excel('Train.xlsx')
df_test = pd.read_excel('testFinal.xlsx')

In [2]:
df = df.iloc[: , 1:]
df_test = df_test.iloc[: , 1:]
df.head()

,id,question,qtype
0,19719,What periodical literature does delta air line...,publication
1,15554,Who is the child of ranavalona i's husband?,person
2,974,Is it true jeff_bridges occupation lane chandl...,boolean
3,27610,Which is the operating income for qantas?,number
4,24488,Which cola starts with the letter p,soft drink


In [3]:
df['question'] = df['question'].str.lower()
df_test['question'] = df_test['question'].str.lower()

In [4]:
 df.head()

,id,question,qtype
0,19719,what periodical literature does delta air line...,publication
1,15554,who is the child of ranavalona i's husband?,person
2,974,is it true jeff_bridges occupation lane chandl...,boolean
3,27610,which is the operating income for qantas?,number
4,24488,which cola starts with the letter p,soft drink


In [5]:
df_test.isna().sum()

id           0
question    16
qtype        0
dtype: int64

In [6]:
df.isna().sum()

id          0
question    0
qtype       0
dtype: int64

In [7]:
df_test = df_test[df_test['question'].notna()]

In [8]:
df['qtype'].value_counts()

boolean                    2139
string                     2086
date                       1301
number                     1044
person                      731
                           ... 
train                         1
airship                       1
city council                  1
constellation                 1
phonetic writing system       1
Name: qtype, Length: 1682, dtype: int64

In [9]:
df_test['qtype'].value_counts()

boolean                                           530
string                                            501
date                                              327
number                                            277
omnivore                                          276
                                                 ... 
non-governmental organization                       1
military operation                                  1
pet                                                 1
aspect of history                                   1
organization established by the United Nations      1
Name: qtype, Length: 768, dtype: int64

In [10]:
df['labels'] = pd.factorize(df.qtype)[0] + 1
df.head(10)

,id,question,qtype,labels
0,19719,what periodical literature does delta air line...,publication,1
1,15554,who is the child of ranavalona i's husband?,person,2
2,974,is it true jeff_bridges occupation lane chandl...,boolean,3
3,27610,which is the operating income for qantas?,number,4
4,24488,which cola starts with the letter p,soft drink,5
5,18423,is the right ascension of malin 1 less than 15...,boolean,3
6,21845,what is the complete list of records released ...,collection,6
7,26800,what's mary lou retton's international olympic...,string,7
8,14923,who won the prize at the sequel of the 1885 wi...,person,2
9,18165,is it true that the carbon footprint of the ip...,boolean,3


In [11]:
df['labels'].max()

1682

In [12]:
import torch
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla K80


In [13]:
!pip install transformers

In [14]:
df_new = df[['qtype', 'labels']].copy()
df_new.head(10)
df_new.to_excel("mapping.xlsx")

In [15]:
questions = df.question.values
ques_type = df.labels.values

In [16]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
import numpy as np
ques_combined = np.concatenate([df.question.values, df_test.question.values])
# print(qtypes_combined)
enc_comb = [tokenizer.encode(q, add_special_tokens=True) for q in ques_combined]
print("Maximum Length: ", max([len(sen) for sen in enc_comb]))

Maximum Length:  187


In [17]:
import re
import string
# Create a function to tokenize a set of texts
#As max length of tokenised IDs is 187, we consider to do padding of length of tokens = 192
def preprocessing_for_bert(data):
   
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []
    
    
    for ques in data:
      ques = re.sub('[%s]' % re.escape(string.punctuation), '', str(ques)) 
      encoded_ques = tokenizer.encode_plus(
            text=ques,  
            add_special_tokens=True,        
            max_length=192,                  
            padding='max_length',         
            return_tensors='pt',           
            return_attention_mask=True      
            )
        
        # Add the outputs to the lists
      input_ids.append(encoded_ques.get('input_ids'))
      attention_masks.append(encoded_ques.get('attention_mask'))

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)


    return input_ids, attention_masks

In [18]:

import torch
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(questions, torch.tensor(ques_type), random_state=6500, test_size=0.1)


In [19]:
# from keras.preprocessing.sequence import pad_sequences


token_ids = list(preprocessing_for_bert([questions[0]])[0])
print('Original: ', questions[0])
print('Token IDs: ', token_ids)

# Run function `preprocessing_for_bert` on the train set and the validation set
# print('Tokenizing data...')
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)

Original:  what periodical literature does delta air lines use as a moutpiece?
Token IDs:  [tensor([  101,  2054, 21802,  3906,  2515,  7160,  2250,  3210,  2224,  2004,
         1037,  9587,  4904, 11198,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [20]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

train_labels = torch.tensor(Y_train)
val_labels = torch.tensor(Y_val)

batch_size = 32
# print(type(Y_train))

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


In [21]:
import torch
import torch.nn as nn
from transformers import BertModel

class bertClassifier(nn.Module):
  def __init__(self, freeze_bert=False):
    super(bertClassifier, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-uncased')
    IN, H, OUT = 768,50,1683
    self.classifier = nn.Sequential(nn.Linear(IN, H),
                                    nn.ReLU(),
                                    nn.Dropout(0.3),
                                    nn.Linear(H,OUT))
    if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
                
  def forward(self, inp_ids, att_masks):
    output = self.bert(input_ids=inp_ids, attention_mask=att_masks)
    last_hidden = output[0][:,0,:]
    output = self.classifier(last_hidden)
    return output

In [22]:
from transformers import AdamW, get_linear_schedule_with_warmup
def initialize_model(epochs=4):
  bert_classifier = bertClassifier(freeze_bert=False)
  bert_classifier.to(device)
  optimizer = AdamW(bert_classifier.parameters(), lr=3e-5, eps=1e-8)
  total_steps = len(train_dataloader)*epochs
  scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0, num_training_steps=total_steps)
  return bert_classifier, optimizer, scheduler

In [23]:
import random
import time

# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 100 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:

        # Load batch to GPU
      b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
      with torch.no_grad():
        logits = model(b_input_ids, b_attn_mask)

        # Compute loss
      loss = loss_fn(logits, b_labels)
      val_loss.append(loss.item())

        # Get the predictions
      preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
      accuracy = (preds == b_labels).cpu().numpy().mean() * 100
      val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [24]:
set_seed(60)    
bert_classifier, optimizer, scheduler = initialize_model(epochs=4)
train(bert_classifier, train_dataloader, val_dataloader, epochs=4, evaluation=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   100   |   6.861675   |     -      |     -     |  190.55  
   1    |   200   |   6.018247   |     -      |     -     |  187.62  
   1    |   300   |   5.501185   |     -      |     -     |  187.46  
   1    |   400   |   5.295113   |     -      |     -     |  187.58  
   1    |   500   |   5.015743   |     -      |     -     |  187.56  
   1    |   513   |   5.056369   |     -      |     -     |   23.15  
----------------------------------------------------------------------
   1    |    -    |   5.723328   |  4.564007  |   39.06   |  1004.97 
----------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   2    |   100   |   4.741770   |     -      |     -     |  189.

In [26]:
import torch.nn.functional as F

def bert_predict(model, test_dataloader):
    
    model.eval()
    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device)
         for t in batch)[:2]
        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    return probs

In [27]:
pred = bert_predict(bert_classifier, val_dataloader)

In [28]:
from sklearn.metrics import accuracy_score
y_pred=np.argmax(pred, axis=1)
accuracy_score(val_labels, y_pred)

0.45071193866374587

In [30]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(val_labels, y_pred)

array([[[1825,    0],
        [   1,    0]],

       [[1731,    6],
        [  89,    0]],

       [[1621,   11],
        [   3,  191]],

       ...,

       [[1825,    0],
        [   1,    0]],

       [[1825,    0],
        [   1,    0]],

       [[1825,    0],
        [   1,    0]]])

In [29]:
# Concatenate the train set and the validation set
full_train_data = torch.utils.data.ConcatDataset([train_data, val_data])
full_train_sampler = RandomSampler(full_train_data)
full_train_dataloader = DataLoader(full_train_data, sampler=full_train_sampler, batch_size=32)

# Train the Bert Classifier on the entire training data
set_seed(75)
bert_classifier, optimizer, scheduler = initialize_model(epochs=4)
train(bert_classifier, full_train_dataloader, epochs=4)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   100   |   6.789696   |     -      |     -     |  189.92  
   1    |   200   |   5.930282   |     -      |     -     |  188.09  
   1    |   300   |   5.659377   |     -      |     -     |  187.10  
   1    |   400   |   5.279056   |     -      |     -     |  186.22  
   1    |   500   |   5.014102   |     -      |     -     |  186.08  
   1    |   570   |   4.876880   |     -      |     -     |  129.19  
----------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   2    |   100   |   4.528836   |     -      |     -     |  188.12  
   2    |   200   |   4.505656   |     -      |     -     |  186.11  
   2    |   300   |   4.413540   |     -      |     -     |  186.1

In [33]:
test_inputs, test_masks = preprocessing_for_bert(df_test.question)

# Create the DataLoader for our test set
test_data = TensorDataset(test_inputs, test_masks)
test_sample = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sample, batch_size=32)

In [36]:
probs = bert_predict(bert_classifier, test_dataloader)

# Get predictions from the probabilities
threshold = 0.1
preds = np.where(probs[:, 1] > threshold, 1, 0)

# Number of tweets predicted non-negative
print("Number of tweets predicted non-negative: ", preds.sum())

Number of tweets predicted non-negative:  0
